In [19]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import joblib
import warnings
warnings.filterwarnings('ignore')

In [20]:
# Load data
print("\n[1/7] Loading data...")
file_path = 'data.xlsx'  # Change to your file path
df = pd.read_excel(file_path)
print(f"✓ Data loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")

# Data preprocessing
print("\n[2/7] Preprocessing data...")
if 'Due Date' in df.columns:
    df['Due Date'] = pd.to_datetime(df['Due Date'], errors='coerce')
    df['Days_to_Due'] = (df['Due Date'] - pd.Timestamp.now()).dt.days
    df['Months_to_Due'] = df['Days_to_Due'] / 30
    df['Years_to_Due'] = df['Days_to_Due'] / 365
    df = df.drop('Due Date', axis=1)

# Handle missing values
df = df.fillna(df.median(numeric_only=True))


[1/7] Loading data...
✓ Data loaded: 99,888 rows, 10 columns

[2/7] Preprocessing data...


In [21]:
# Feature Engineering
print("\n[3/7] Engineering features...")

# Create ratio and interaction features
df['Rate_Difference'] = df['Effec. Rate'] - df['Flat Rate']
df['Rental_to_Amount_Ratio'] = df['Net Rental'] / (df['Facility amount'] + 1)
df['Amount_per_Tenor'] = df['Facility amount'] / (df['Tenor'] + 1)
df['Rental_per_Tenor'] = df['Net Rental'] / (df['Tenor'] + 1)
df['Arrears_Rate'] = df['No of Rental in arrears'] / (df['Tenor'] + 1)
df['Total_Payment'] = df['Net Rental'] * df['Tenor']
df['Payment_Capacity'] = df['Facility amount'] / (df['Total_Payment'] + 1)
df['Risk_Score'] = df['No of Rental in arrears'] * df['Effec. Rate'] / 100
df['Age_Tenor_Interaction'] = df['Age'] * df['Tenor']
df['Amount_Rate_Interaction'] = df['Facility amount'] * df['Effec. Rate'] / 100
df['Arrears_Amount'] = df['No of Rental in arrears'] * df['Net Rental']

# Logarithmic features for skewed data
df['Log_Facility_Amount'] = np.log1p(df['Facility amount'])
df['Log_Net_Rental'] = np.log1p(df['Net Rental'])

# Squared features
df['Tenor_Squared'] = df['Tenor'] ** 2
df['Age_Squared'] = df['Age'] ** 2
df['Arrears_Squared'] = df['No of Rental in arrears'] ** 2

# Polynomial features for key variables
df['Rate_Squared'] = df['Effec. Rate'] ** 2
df['Rate_Cubed'] = df['Effec. Rate'] ** 3

print(f"✓ Features created: {df.shape[1]} total features")


[3/7] Engineering features...
✓ Features created: 30 total features


In [22]:
# Separate features and targets
target_columns = ['Impairment', '1 yr ECL']
feature_columns = [col for col in df.columns if col not in target_columns]

X = df[feature_columns]
y_impairment = df['Impairment']
y_ecl = df['1 yr ECL']

# Split data (80-20 split for large dataset)
X_train, X_test, y_imp_train, y_imp_test = train_test_split(
    X, y_impairment, test_size=0.2, random_state=42
)
_, _, y_ecl_train, y_ecl_test = train_test_split(
    X, y_ecl, test_size=0.2, random_state=42
)

print(f"✓ Train set: {len(X_train):,} samples | Test set: {len(X_test):,} samples")

✓ Train set: 79,910 samples | Test set: 19,978 samples


In [33]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
try:
    joblib.dump(scaler, 'scaler_advanced.pkl')
    print("Scaler saved successfully ✅")
except Exception as e:
    print("Error saving scaler:", e)

# Define advanced models with optimized hyperparameters
print("\n[4/7] Initializing advanced models...")

models = {
    'Random Forest': RandomForestRegressor(
        n_estimators=200,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1
    ),
    'XGBoost': XGBRegressor(
        n_estimators=200,
        max_depth=10,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ),
    'LightGBM': LGBMRegressor(
        n_estimators=200,
        max_depth=15,
        learning_rate=0.1,
        num_leaves=50,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    ),
    'CatBoost': CatBoostRegressor(
        iterations=200,
        depth=10,
        learning_rate=0.1,
        random_state=42,
        verbose=0
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=200,
        max_depth=10,
        learning_rate=0.1,
        subsample=0.8,
        random_state=42
    ),
    'Ridge (Polynomial)': Ridge(alpha=1.0)
}

print(f"✓ {len(models)} models initialized")

Scaler saved successfully ✅

[4/7] Initializing advanced models...
✓ 6 models initialized


In [24]:
# Evaluation function
def evaluate_model(y_true, y_pred, model_name, target_name):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100
    
    accuracy_percent = r2 * 100
    status = "✓ TARGET ACHIEVED" if accuracy_percent >= 95 else "⚠ Below target"
    
    print(f"\n  {model_name} - {target_name}:")
    print(f"    R² Score: {r2:.6f} ({accuracy_percent:.2f}%) {status}")
    print(f"    RMSE: {rmse:.4f}")
    print(f"    MAE: {mae:.4f}")
    print(f"    MAPE: {mape:.2f}%")
    
    return r2, rmse, mae, mape

# Store results
results = []

In [26]:
print("\n[5/7] Training models (this may take a few minutes)...")

for model_name, model in models.items():
    print(f"\n{'='*70}")
    print(f"Training: {model_name}")
    print(f"{'='*70}")
    
    # IMPAIRMENT
    print(f"\n--- IMPAIRMENT PREDICTION ---")
    model_imp = model.__class__(**model.get_params()) if hasattr(model, 'get_params') else model
    model_imp.fit(X_train_scaled, y_imp_train)
    y_imp_pred = model_imp.predict(X_test_scaled)
    r2_imp, rmse_imp, mae_imp, mape_imp = evaluate_model(
        y_imp_test, y_imp_pred, model_name, "Impairment"
    )
    
    # Save model
    filename = f"{model_name.replace(' ', '_').replace('(', '').replace(')', '').lower()}_impairment.pkl"
    joblib.dump(model_imp, filename)
    print(f"    Model saved: {filename}")
    
    # 1 YR ECL
    print(f"\n--- 1 YR ECL PREDICTION ---")
    model_ecl = model.__class__(**model.get_params()) if hasattr(model, 'get_params') else model
    model_ecl.fit(X_train_scaled, y_ecl_train)
    y_ecl_pred = model_ecl.predict(X_test_scaled)
    r2_ecl, rmse_ecl, mae_ecl, mape_ecl = evaluate_model(
        y_ecl_test, y_ecl_pred, model_name, "1 yr ECL"
    )
    
    # Save model
    filename = f"{model_name.replace(' ', '_').replace('(', '').replace(')', '').lower()}_ecl.pkl"
    joblib.dump(model_ecl, filename)
    print(f"    Model saved: {filename}")
    
    results.append({
        'Model': model_name,
        'Target': 'Impairment',
        'R² Score': r2_imp,
        'Accuracy %': r2_imp * 100,
        'RMSE': rmse_imp,
        'MAE': mae_imp,
        'MAPE %': mape_imp,
        'Status': '✓ Achieved' if r2_imp * 100 >= 95 else '⚠ Below 95%'
    })
    results.append({
        'Model': model_name,
        'Target': '1 yr ECL',
        'R² Score': r2_ecl,
        'Accuracy %': r2_ecl * 100,
        'RMSE': rmse_ecl,
        'MAE': mae_ecl,
        'MAPE %': mape_ecl,
        'Status': '✓ Achieved' if r2_ecl * 100 >= 95 else '⚠ Below 95%'
    })


[5/7] Training models (this may take a few minutes)...

Training: Random Forest

--- IMPAIRMENT PREDICTION ---

  Random Forest - Impairment:
    R² Score: 0.915779 (91.58%) ⚠ Below target
    RMSE: 53717.5022
    MAE: 5419.9504
    MAPE: 195339157562795.75%
    Model saved: random_forest_impairment.pkl

--- 1 YR ECL PREDICTION ---

  Random Forest - 1 yr ECL:
    R² Score: 0.879151 (87.92%) ⚠ Below target
    RMSE: 7229.0728
    MAE: 2709.1375
    MAPE: 284742932561131.56%
    Model saved: random_forest_ecl.pkl

Training: XGBoost

--- IMPAIRMENT PREDICTION ---

  XGBoost - Impairment:
    R² Score: 0.978224 (97.82%) ✓ TARGET ACHIEVED
    RMSE: 27314.2927
    MAE: 4845.7441
    MAPE: 157433259095566.12%
    Model saved: xgboost_impairment.pkl

--- 1 YR ECL PREDICTION ---

  XGBoost - 1 yr ECL:
    R² Score: 0.926418 (92.64%) ⚠ Below target
    RMSE: 5640.8659
    MAE: 2275.5480
    MAPE: 177670187417804.09%
    Model saved: xgboost_ecl.pkl

Training: LightGBM

--- IMPAIRMENT PREDICTIO

In [27]:
# Create ensemble stacking model
print(f"\n{'='*70}")
print("Training ENSEMBLE STACKING MODEL (Meta-Model)")
print(f"{'='*70}")

estimators = [
    ('rf', RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
    ('xgb', XGBRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
    ('lgb', LGBMRegressor(n_estimators=200, random_state=42, verbose=-1, n_jobs=-1))
]


Training ENSEMBLE STACKING MODEL (Meta-Model)


In [28]:
# Stacking for Impairment
print(f"\n--- STACKING ENSEMBLE - IMPAIRMENT ---")
stacking_imp = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(),
    cv=5
)
stacking_imp.fit(X_train_scaled, y_imp_train)
y_imp_pred_stack = stacking_imp.predict(X_test_scaled)
r2_imp_stack, rmse_imp_stack, mae_imp_stack, mape_imp_stack = evaluate_model(
    y_imp_test, y_imp_pred_stack, "Stacking Ensemble", "Impairment"
)
joblib.dump(stacking_imp, 'stacking_ensemble_impairment.pkl')
print(f"    Model saved: stacking_ensemble_impairment.pkl")


--- STACKING ENSEMBLE - IMPAIRMENT ---

  Stacking Ensemble - Impairment:
    R² Score: 0.992412 (99.24%) ✓ TARGET ACHIEVED
    RMSE: 16124.2605
    MAE: 4982.4596
    MAPE: 154303801377173.34%
    Model saved: stacking_ensemble_impairment.pkl


In [29]:
print(f"\n--- STACKING ENSEMBLE - 1 YR ECL ---")
stacking_ecl = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(),
    cv=5
)
stacking_ecl.fit(X_train_scaled, y_ecl_train)
y_ecl_pred_stack = stacking_ecl.predict(X_test_scaled)
r2_ecl_stack, rmse_ecl_stack, mae_ecl_stack, mape_ecl_stack = evaluate_model(
    y_ecl_test, y_ecl_pred_stack, "Stacking Ensemble", "1 yr ECL"
)
joblib.dump(stacking_ecl, 'stacking_ensemble_ecl.pkl')
print(f"    Model saved: stacking_ensemble_ecl.pkl")


--- STACKING ENSEMBLE - 1 YR ECL ---

  Stacking Ensemble - 1 yr ECL:
    R² Score: 0.928455 (92.85%) ⚠ Below target
    RMSE: 5562.2574
    MAE: 1899.7495
    MAPE: 164874526842369.38%
    Model saved: stacking_ensemble_ecl.pkl


In [30]:
results.append({
    'Model': 'Stacking Ensemble',
    'Target': 'Impairment',
    'R² Score': r2_imp_stack,
    'Accuracy %': r2_imp_stack * 100,
    'RMSE': rmse_imp_stack,
    'MAE': mae_imp_stack,
    'MAPE %': mape_imp_stack,
    'Status': '✓ Achieved' if r2_imp_stack * 100 >= 95 else '⚠ Below 95%'
})
results.append({
    'Model': 'Stacking Ensemble',
    'Target': '1 yr ECL',
    'R² Score': r2_ecl_stack,
    'Accuracy %': r2_ecl_stack * 100,
    'RMSE': rmse_ecl_stack,
    'MAE': mae_ecl_stack,
    'MAPE %': mape_ecl_stack,
    'Status': '✓ Achieved' if r2_ecl_stack * 100 >= 95 else '⚠ Below 95%'
})


In [31]:
# Results summary
print("\n[6/7] Generating results summary...")
print(f"\n{'='*70}")
print("COMPLETE RESULTS SUMMARY")
print(f"{'='*70}")

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(['Target', 'Accuracy %'], ascending=[True, False])

print("\n", results_df.to_string(index=False))

# Save results
results_df.to_csv('advanced_model_results.csv', index=False)
print(f"\n✓ Results saved: advanced_model_results.csv")

# Best models
print(f"\n{'='*70}")
print("BEST PERFORMING MODELS")
print(f"{'='*70}")

best_imp = results_df[results_df['Target'] == 'Impairment'].iloc[0]
best_ecl = results_df[results_df['Target'] == '1 yr ECL'].iloc[0]

print(f"\n🏆 BEST FOR IMPAIRMENT:")
print(f"   Model: {best_imp['Model']}")
print(f"   Accuracy: {best_imp['Accuracy %']:.2f}%")
print(f"   Status: {best_imp['Status']}")

print(f"\n🏆 BEST FOR 1 YR ECL:")
print(f"   Model: {best_ecl['Model']}")
print(f"   Accuracy: {best_ecl['Accuracy %']:.2f}%")
print(f"   Status: {best_ecl['Status']}")

# Models achieving 95%+
achieving_target = results_df[results_df['Accuracy %'] >= 95]
if len(achieving_target) > 0:
    print(f"\n✓ {len(achieving_target)} model(s) achieved 95%+ accuracy:")
    for _, row in achieving_target.iterrows():
        print(f"   • {row['Model']} ({row['Target']}): {row['Accuracy %']:.2f}%")
else:
    print("\n⚠ No models achieved 95%+ accuracy yet.")
    print("   Consider:")
    print("   1. Check if Impairment/ECL are highly correlated with input features")
    print("   2. Add domain-specific features")
    print("   3. Check for data quality issues")
    print("   4. Try hyperparameter tuning (GridSearchCV)")

print(f"\n[7/7] Process complete!")
print(f"\n{'='*70}")
print("ALL MODELS TRAINED AND EXPORTED SUCCESSFULLY!")
print(f"{'='*70}")

print("\n📦 EXPORTED FILES:")
print(f"   • {len(results_df)} model files (.pkl)")
print("   • scaler_advanced.pkl")
print("   • advanced_model_results.csv")

print("\n💡 USAGE EXAMPLE:")
print("""
# Load model and scaler
import joblib
scaler = joblib.load('scaler_advanced.pkl')
model = joblib.load('catboost_impairment.pkl')  # Use best model

# Prepare new data (must match training features exactly)
new_data = pd.DataFrame({...})  # Add all features

# Scale and predict
new_data_scaled = scaler.transform(new_data)
prediction = model.predict(new_data_scaled)
print(f'Prediction: {prediction[0]}')
""")


[6/7] Generating results summary...

COMPLETE RESULTS SUMMARY

              Model     Target  R² Score  Accuracy %         RMSE          MAE       MAPE %      Status
 Stacking Ensemble   1 yr ECL  0.928455   92.845482  5562.257421  1899.749483 1.648745e+14 ⚠ Below 95%
           XGBoost   1 yr ECL  0.926418   92.641831  5640.865915  2275.548013 1.776702e+14 ⚠ Below 95%
           XGBoost   1 yr ECL  0.926418   92.641831  5640.865915  2275.548013 1.776702e+14 ⚠ Below 95%
 Gradient Boosting   1 yr ECL  0.921338   92.133818  5832.340819  2290.011066 1.732999e+14 ⚠ Below 95%
     Random Forest   1 yr ECL  0.879151   87.915091  7229.072827  2709.137519 2.847429e+14 ⚠ Below 95%
     Random Forest   1 yr ECL  0.879151   87.915091  7229.072827  2709.137519 2.847429e+14 ⚠ Below 95%
          LightGBM   1 yr ECL  0.844245   84.424495  8206.953551  3487.998092 3.737818e+14 ⚠ Below 95%
          LightGBM   1 yr ECL  0.844245   84.424495  8206.953551  3487.998092 3.737818e+14 ⚠ Below 95%
        